## Text2Video Generation, using ByteDance's AnimateDiff-Lightning.

Created by: [Alex Jenkins](https://alexj.io)

Professors: [Dr. Francesco Fedele](https://ce.gatech.edu/directory/person/francesco-fedele) and [Dr. Mark Leibert](https://lmc.gatech.edu/people/person/mark-leibert)

### MAKE SURE TO RUN WITH THE T4 GPU UNDER RUNTIME OPTIONS!

#### Version: 02/23/2025

Model Card: [View here](https://huggingface.co/ByteDance/AnimateDiff-Lightning).

Copyright (c) 2025, [Georgia Institute of Technology](https://www.gatech.edu).


In [ ]:
#@title 1). Install packages to prepare to download the model
#@markdown ⬅️ Press to install and prepare the model.
#@markdown Settings will appear once finished.

# Install necessary libraries
!pip install torch diffusers huggingface_hub safetensors ipywidgets

import torch
from diffusers import AnimateDiffPipeline, MotionAdapter, EulerDiscreteScheduler
from diffusers.utils import export_to_gif
from huggingface_hub import hf_hub_download
from safetensors.torch import load_file
import ipywidgets as widgets
from IPython.display import display
import gc

# Define sliders for generation quality settings
guidance_scale = widgets.FloatSlider(value=5.0, min=1.0, max=10.0, step=0.5, description="Guidance Scale")
num_inference_steps = widgets.IntSlider(value=4, min=1, max=8, step=1, description="Inference Steps")
height = widgets.IntSlider(value=256, min=128, max=512, step=64, description="Height")
width = widgets.IntSlider(value=256, min=128, max=512, step=64, description="Width")

# Display sliders for user input
display(guidance_scale, num_inference_steps, height, width)

In [ ]:
#@title 2). Run the model (ETA: ~1 minute per video)
#@markdown ⬅️ Press to run model, you will be asked for the prompt once loaded.

#@markdown **Note:** This may take about 5 minutes on the first run to download the model

# Set device and dtype
device = "cuda" if torch.cuda.is_available() else "cpu"
dtype = torch.float16 if device == "cuda" else torch.float32

# Default configuration
repo = "ByteDance/AnimateDiff-Lightning"
step = 4  # Default step value, can be changed
ckpt = f"animatediff_lightning_{step}step_diffusers.safetensors"
base = "emilianJR/epiCRealism"

# Load the motion adapter
adapter = MotionAdapter().to(device, dtype)
adapter.load_state_dict(load_file(hf_hub_download(repo, ckpt), device=device))

# Load the pipeline
pipe = AnimateDiffPipeline.from_pretrained(
    base,
    motion_adapter=adapter,
    torch_dtype=dtype,
    low_cpu_mem_usage=True,  # Optimize memory usage
).to(device)

# Use a faster scheduler
pipe.scheduler = EulerDiscreteScheduler.from_config(
    pipe.scheduler.config,
    timestep_spacing="trailing",
    beta_schedule="linear"
)

# Get user input for prompt
prompt = input("Enter your text prompt:\n")

# Generate animation
output = pipe(
    prompt=prompt,
    guidance_scale=guidance_scale.value,
    num_inference_steps=num_inference_steps.value,
    height=height.value,
    width=width.value,
)

# Export animation to GIF
export_to_gif(output.frames[0], "animation.gif")

# Display the GIF
from IPython.display import Image
display(Image(filename="animation.gif"))

# Free memory after execution
del pipe
del adapter
torch.cuda.empty_cache()
gc.collect()